In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [2]:
response = requests.get("https://www.cermati.com/karir")
soup = BeautifulSoup(response.text, 'lxml')

In [3]:
def scrape_info(url):
    info = dict()
    description = []
    qualfication = []
    resp = requests.get(url)
    job_info = BeautifulSoup(resp.text, 'lxml')
    
    info['title'] = job_info.main.h1.text
    info['location']=job_info.main.ul.li.span.text
    
    try:
        soup2 = BeautifulSoup(str(job_info.find('div', {'itemprop':'responsibilities'})), 'lxml')
        for li in  soup2.ul.contents:
            description.append(li.text.replace('\u00a0', '').replace('\u2019', "'").strip('\xa0'))
        info['description']=description
    except AttributeError as e:
        print("No description")    
    
    try:
        soup3 = BeautifulSoup(str(job_info.find('div', {'itemprop':'qualifications'})), 'lxml')
        for li in  soup3.ul.contents:
            qualfication.append(li.text.replace('\u00a0', '').replace('\u2019', "'").strip('\xa0'))
        info['qualfication']=qualfication
    except AttributeError as e:
        print("No qualfication")
    
    return info
    

In [ ]:
dept_job = dict()
for tab in range(6):
    dept = soup.find_all('div', {'id': 'tab'+str(tab)})
    soup_dept = BeautifulSoup(str(dept), 'lxml')
    
    print("fetching: " + soup_dept.h4.text)

    job_info = []
    url = []
    jobs = soup_dept.find_all('div', {'class': 'clickable-row row'})
    for j in jobs:
        job_url = BeautifulSoup(str(j), 'lxml')
        url.append(job_url.a.get('href').split()[0])
        

    with Pool(os.cpu_count()) as p:
        job_info = p.map(scrape_info, url)
    
    dept_job[soup_dept.h4.text]=job_info

fetching: Engineering


In [31]:
with open('result.json', 'w') as fp:
    json.dump(dept_job, fp)